In [5]:
import pandas as pd
import numpy as np

from dsgd.DSClassifierMultiQ import DSClassifierMultiQ

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


In [6]:
df = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/train.csv")
df = df.drop(columns=['index'])
print("Dataframe shape: ", df.shape)
df.head()
test = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/test.csv")
test = test.drop(columns=['index'])
print("Dataframe shape: ", test.shape)

Dataframe shape:  (1400, 27)
Dataframe shape:  (400, 27)


In [7]:
y = df['label']
X = df.drop(columns=['label'])
y = y.to_numpy()
X = X.to_numpy()    
y_test = test['label']
X_test = test.drop(columns=['label'])
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [8]:
DSCs = {}
time = {}
accuracys = {}
learning_rates = [0.0001, 0.001, 0.01, 0.1]
batch_sizes = [2000, 3000, 4000]
min_dl = [1e-7, 1e-6, 1e-5,]
num_rules = [3, 5, 7,]
df_results = pd.DataFrame(columns=['lr', 'bs', 'mdl', 'nr', 'accuracy', 'f1_micro', 'f1_macro'])
for lr in learning_rates:
    for bs in batch_sizes:
        for mdl in min_dl:
            for nr in num_rules:
                DSC = DSClassifierMultiQ(2, min_iter=20, max_iter=500, debug_mode=True, lr=lr, batch_size=bs,
                                lossfn="MSE", num_workers=1, min_dloss=mdl, precompute_rules=True)
                print(f"Training DSC with lr={lr}, bs={bs}, mdl={mdl}, nr={nr}")
                losses, epoch, dt = DSC.fit(X, y, add_single_rules=True,
                            single_rules_breaks=nr, add_mult_rules=False,
                                column_names=df.columns[:-1], print_every_epochs=1, print_final_model=False)
                y_pred = DSC.predict(X_test)
                acc = accuracy_score(y_test, y_pred)
                f1_micro = f1_score(y_test, y_pred, average='micro')
                f1_macro = f1_score(y_test, y_pred, average='macro')
                print(f"Accuracy: {acc}, F1 micro: {f1_micro}, F1 macro: {f1_macro}")
                new_row = pd.DataFrame({
                    'lr': [lr], 'bs': [bs], 'mdl': [mdl], 'nr': [nr],
                    'accuracy': [acc], 'f1_micro': [f1_micro], 'f1_macro': [f1_macro]
                })
                df_results = pd.concat([df_results, new_row])
                

df_results.to_csv("results2.csv", index=False)

Training DSC with lr=0.0001, bs=2000, mdl=1e-07, nr=3
Optimization started
Processing epoch	500	0.2173	
Training time: 1178.00s, epochs: 500

Least training loss reached: 0.217
Accuracy: 0.6475, F1 micro: 0.6475, F1 macro: 0.6447803495512519
Training DSC with lr=0.0001, bs=2000, mdl=1e-07, nr=5
Optimization started
Processing epoch	1	1.0000	

C:\Users\Daniel\AppData\Local\Temp\ipykernel_5104\4004509887.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, new_row])


Processing epoch	500	0.2218	
Training time: 1147.55s, epochs: 500

Least training loss reached: 0.222
Accuracy: 0.6325, F1 micro: 0.6325, F1 macro: 0.6324793269621416
Training DSC with lr=0.0001, bs=2000, mdl=1e-07, nr=7
Optimization started
Processing epoch	500	0.2168	
Training time: 1128.83s, epochs: 500

Least training loss reached: 0.217
Accuracy: 0.6275, F1 micro: 0.6275, F1 macro: 0.6274790456963204
Training DSC with lr=0.0001, bs=2000, mdl=1e-06, nr=3
Optimization started
Processing epoch	500	0.2209	
Training time: 1104.04s, epochs: 500

Least training loss reached: 0.221
Accuracy: 0.6575, F1 micro: 0.6575, F1 macro: 0.6572407883461868
Training DSC with lr=0.0001, bs=2000, mdl=1e-06, nr=5
Optimization started
Processing epoch	500	0.2168	
Training time: 1097.14s, epochs: 500

Least training loss reached: 0.217
Accuracy: 0.66, F1 micro: 0.66, F1 macro: 0.6598639455782312
Training DSC with lr=0.0001, bs=2000, mdl=1e-06, nr=7
Optimization started
Processing epoch	500	0.2094	
Trainin

In [11]:
df_results = pd.read_csv('results2.csv')
df_results = df_results.sort_values(by='accuracy', ascending=False)
df_results.head(5)

,lr,bs,mdl,nr,accuracy,f1_micro,f1_macro
5,0.0001,2000,1.000000e-06,7,0.6775,0.6775,0.673366
26,0.0001,4000,1.000000e-05,7,0.6675,0.6675,0.666748
16,0.0001,3000,1.000000e-05,5,0.6650,0.6650,0.664790
10,0.0001,3000,1.000000e-07,5,0.6650,0.6650,0.662264
4,0.0001,2000,1.000000e-06,5,0.6600,0.6600,0.659864
